In [ ]:
# default_exp core

# bayes_coin_flip

> Implementation of bayesian reasoning on a coin-flip like scenario with finite list $\{p\}$ of possible latent parameter $p$

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from lib.lib_nicefloat import *
from std_init import *
jax.config.update('jax_platform_name', 'cpu')

ja=lambda x: jax.numpy.array(x)

def coin_model(real_hidden_phi=0.5 ,observation_count=10):
    """
    a model of a coin: simulates coin tosses.
    
    """
    observations = jax.numpy.array(numpy.random.random(observation_count)<real_hidden_phi,dtype=jax.numpy.int32).reshape(observation_count,1)
    return observations


# note that the convention is to put phis in column headers column headers
phi_set = ja([0.2,0.5,0.8])
observation_single_1 = ja([[1]]) # note: need a categorical variable here - and only "1" or "0"
observation_single_0 = ja([[0]]) # note: need a categorical variable here - and only "1" or "0"
observation_small_set_0 = ja([[0],[0]])
observation_small_set_balanced = ja([[0],[1]])
observation_small_set_1 = ja([[1],[1]])

# note that the convention is to put observations in rows of the table.
observation_large_set_neutral = ja([[1],[1],[1],[1],[0],[0],[0],[0]])
observation_large_set_biased = ja([[1],[1],[1],[1],[0],[0]])

In [ ]:
#export

# like with phi, we put prior and posterior beliefs in column headers (columns, or 1D-only arrays)

# no bias belief: we think that the coin is not biased.
prior_belief_set_fair = ja([0.1,0.8,0.1])
pb(prior_belief_set_fair)



# high bias belief: we think that the coin is biased.
prior_belief_set_bias_0 = ja([0.8,0.1,0.1])
pb(prior_belief_set_bias_0)



# high bias belief: we think that the coin is biased towards ones.
prior_belief_set_bias_1 = jax.numpy.array([0.1,0.1,0.8])
pb(prior_belief_set_bias_1)

prior_belief_set_uninformative = jax.numpy.array([0.3333,0.3333,0.3333])
pb(prior_belief_set_uninformative)

[    333mR,     333mR,     333mR, ]

In [ ]:
#export

def likelihood_vector_fn(observations,phis):
    assert observations.shape[1]==1, f"only 1 feature per observation supported. Supplied {observations.shape[1]=}"
    assert len(phis.shape)==1, f"set of phi must be in a 1D vector. Supplied {phis.shape=}, expected phis.shape=(n,)"
    result = jax.numpy.einsum('ox,p->op',(observations==1),phis) + jax.numpy.einsum('ox,p->op',(observations==0),(1-phis))
    return result

def likelihood_observation_set_fn(observations,phi):
    result = jax.numpy.prod(likelihood_vector_fn(observations,phi),axis=0)
    return result

def marginal_fn(observations, likelihood_function, phi_set, prior_belief_set):
    # note that the likehood_function must be the observation_set likehood function (one that computes likehood of all observations simultanousely)
    result = jax.numpy.sum(prior_belief_set*likelihood_function(observations=observations,phi=phi_set),axis=0)
    return result

# scalar version -- simple.
def BayesianBeliefUpdate_singlePhi(
        this_phi, this_phi_prior_belief,
        phi_set, phi_prior_belief_set,
        observations,
        likelihood_fn,
        marginal_probability_fn
        ):
    result = this_phi_prior_belief * ( likelihood_fn(observations,this_phi) / marginal_probability_fn(observations, likelihood_fn, phi_set, phi_prior_belief_set) )
    return result

def BayesianBeliefUpdate(
    phi_set, phi_prior_belief_set, observations,
    likelihood_fn, marginal_probability_fn,
    ):
    posterior_belief_set = []
    # I choose to do the iteration here, because it is still confusing to write an update function for each phi taking all phis . . .
    # still, it's probably possible to refactor the marginal_probability_fn to not compute it multiple times -- at the cost of making it more convoluted to implement.
    for phi_index in range(len(phi_set)):
        UpdatedBelief = BayesianBeliefUpdate_singlePhi(
            this_phi=ja(phi_set[phi_index]).reshape(-1,),                # for this phi,
            this_phi_prior_belief=phi_prior_belief_set[phi_index],       # with such prior belief
            phi_set=phi_set, phi_prior_belief_set=phi_prior_belief_set,  # out of this phi set
            observations=observations,                                   # taking in these new observations
            likelihood_fn=likelihood_observation_set_fn,                 #the likehood function for the model
            marginal_probability_fn=marginal_probability_fn)             #the marginal probability function to call
        posterior_belief_set.append(UpdatedBelief[0])
    posterior_belief_set = ja(posterior_belief_set).T

    return posterior_belief_set


def BayesianBeliefUpdateReport(
                        phi_set = phi_set,
                        phi_prior_belief_set=prior_belief_set_uninformative,
                        observations=observation_small_set_balanced,
                        posterior_belief_set=prior_belief_set_uninformative):
    if len(observations)<5:
        print('for observations {D}: \n'f'{pd(observations)}')
    else:
        observation_count = len(observations)
        print(f'for {len(observations)} observations, ')
        print(f'{jax.numpy.sum(observations==0)/observation_count*100:0.1f}% of observations are D==0')
        print(f'{jax.numpy.sum(observations==1)/observation_count*100:0.1f}% of observations are D==1')
        print(f'')
    print('for parameter set      {φ}  = 'f'{pf(phi_set)}')
    print('and prior belief set b({φ}) = 'f'{pb(phi_prior_belief_set)}\n')
    for phi_index in range(len(phi_set)):
        belief_change = posterior_belief_set[phi_index]-phi_prior_belief_set[phi_index]
        print(f'for model parameter {pf(phi_set[phi_index])} with PriorBelief = {pb(phi_prior_belief_set[phi_index])}, UpdatedBelief = {pb(posterior_belief_set[phi_index])}, change of {float(belief_change*1e3):+5.0f}mR ')
    print(f'\n')
    print('                       {φ}  = 'f'{pf(phi_set)}')
    print('posterior belief set b({φ}) = 'f'{pb(posterior_belief_set)}')

In [ ]:

this_observation_set = observation_small_set_0
this_phi_prior_belief_set = prior_belief_set_uninformative

posterior_belief_set=BayesianBeliefUpdate(
                        phi_set = phi_set,
                        phi_prior_belief_set=this_phi_prior_belief_set,
                        observations=this_observation_set,
                        likelihood_fn=likelihood_observation_set_fn,
                        marginal_probability_fn = marginal_fn)

BayesianBeliefUpdateReport(
                        phi_set = phi_set,
                        phi_prior_belief_set=this_phi_prior_belief_set,
                        observations=this_observation_set,
                        posterior_belief_set=posterior_belief_set)

for observations {D}: 
[[D=  0  , ],
 [D=  0  , ]]

for parameter set      {φ}  = [φ=0.200  , φ=0.500  , φ=0.800  , ]
and prior belief set b({φ}) = [    333mR,     333mR,     333mR, ]

for model parameter φ=0.200  ,  with PriorBelief =     333mR, , UpdatedBelief =     688mR, , change of  +355mR 
for model parameter φ=0.500  ,  with PriorBelief =     333mR, , UpdatedBelief =     269mR, , change of   -64mR 
for model parameter φ=0.800  ,  with PriorBelief =     333mR, , UpdatedBelief =      43mR, , change of  -290mR 


                       {φ}  = [φ=0.200  , φ=0.500  , φ=0.800  , ]
posterior belief set b({φ}) = [    688mR,     269mR,      43mR, ]


In [ ]:
pd(observation_large_set_biased)

[[D=  1  , ],
 [D=  1  , ],
 [D=  1  , ],
 [D=  1  , ],
 [D=  0  , ],
 [D=  0  , ]]

In [ ]:
pd(coin_model(real_hidden_phi=0.5, observation_count =5))

[[D=  1  , ],
 [D=  1  , ],
 [D=  1  , ],
 [D=  0  , ],
 [D=  1  , ]]

In [ ]:
# we will put the phi under consideration in columns of the table

pf(phi_set)

[φ=0.200  , φ=0.500  , φ=0.800  , ]

In [ ]:
# when the coin is fair, the chance of getting (1) is 1/2
pl(likelihood_observation_set_fn(observations=observation_single_1,phi=ja([0.5])))


    500mΩ, 

In [ ]:
assert likelihood_observation_set_fn(observations=observation_single_1,phi=ja([0.5]))==0.5

In [ ]:
pl(likelihood_vector_fn(observation_large_set_neutral,phi_set))

[[    200mΩ,     500mΩ,     800mΩ, ],
 [    200mΩ,     500mΩ,     800mΩ, ],
 [    200mΩ,     500mΩ,     800mΩ, ],
 [    200mΩ,     500mΩ,     800mΩ, ],
 [    800mΩ,     500mΩ,     200mΩ, ],
 [    800mΩ,     500mΩ,     200mΩ, ],
 [    800mΩ,     500mΩ,     200mΩ, ],
 [    800mΩ,     500mΩ,     200mΩ, ]]

In [ ]:
#export

assert jax.numpy.allclose(
    a= likelihood_observation_set_fn(observation_large_set_neutral,phi_set),
    b = jax.numpy.array([0.00065536, 0.00390625, 0.00065536], dtype=jax.numpy.float32)
)==True

In [ ]:
# hide
# from nbdev.export import notebook2script; notebook2script()